In [16]:
import boto3
import pandas as pd

In [2]:


s3 = boto3.client('s3')
s3.download_file('data-handling-public', 'products.csv',
                 'cred/products.csv')

In [17]:
aws_bucket ='data-handling-public'
s3_key= 'products.csv'
local_path= 'cred/products.csv'

In [18]:

response = s3.get_object(Bucket=aws_bucket, Key=s3_key)

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    products = pd.read_csv(response.get("Body"))
    print(products)
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200
      Unnamed: 0                                 product_name product_price  \
0              0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1              1          Tiffany's World Day Out At The Park        £12.99   
2              2          Tiffany's World Pups Picnic Playset         £7.00   
3              3     Tiffany's World Wildlife Park Adventures        £12.99   
4              4                      Cosatto Cosy Dolls Pram        £30.00   
...          ...                                          ...           ...   
1848        1848    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1849        1849    Goodmans Qi Wireless Charging Phone Stand        £15.00   
1850        1850                     RAC 12V Wet & Dry Vacuum        £18.00   
1851        1851    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1852        1852               Beldray Telescopic Ladder 2.9m        £69.00   

   